In [184]:
import pandas as pd
import numpy as np
import random
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandasql as ps

#### 1. 处理用户app使用习惯维度的feature

读取app_categories.csv

In [185]:
df_app_categories = pd.read_csv('dataset/pre/app_categories.csv')
print(df_app_categories.info())
print(df_app_categories.describe())
len(df_app_categories.appCategory.unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217041 entries, 0 to 217040
Data columns (total 2 columns):
appID          217041 non-null int64
appCategory    217041 non-null int64
dtypes: int64(2)
memory usage: 3.3 MB
None
               appID    appCategory
count  217041.000000  217041.000000
mean   137220.306472     161.856133
std    105340.872671     157.746571
min        14.000000       0.000000
25%     54585.000000       0.000000
50%    111520.000000     106.000000
75%    195882.000000     301.000000
max    433269.000000     503.000000


28

读取 user_app_actions.csv

In [186]:
df_user_appactions = pd.read_csv('dataset/pre/user_app_actions.csv')
print(df_user_appactions.info())
print(df_user_appactions.describe())
df_user_appactions.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6003471 entries, 0 to 6003470
Data columns (total 3 columns):
userID         int64
installTime    int64
appID          int64
dtypes: int64(3)
memory usage: 137.4 MB
None
             userID   installTime         appID
count  6.003471e+06  6.003471e+06  6.003471e+06
mean   1.403576e+06  1.483695e+05  5.125716e+03
std    8.074011e+05  9.097316e+04  2.118739e+04
min    1.000000e+00  1.000000e+04  3.540000e+02
25%    7.055445e+05  6.164500e+04  5.890000e+02
50%    1.404560e+06  1.408000e+05  8.270000e+02
75%    2.100952e+06  2.320160e+05  1.969000e+03
max    2.805117e+06  3.023590e+05  4.332670e+05


,userID,installTime,appID
1189493,560310,251901,608
572290,271897,111403,818
3514642,1641340,12345,1824
5547405,2590679,21951,455
2962808,1386246,30756,710


读取 user_installedapps.csv

In [187]:
df_user_installedapps = pd.read_csv('dataset/pre/user_installedapps.csv')
print(df_user_installedapps.info())
print(df_user_installedapps.describe())
df_user_installedapps.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84039009 entries, 0 to 84039008
Data columns (total 2 columns):
userID    int64
appID     int64
dtypes: int64(2)
memory usage: 1.3 GB
None
             userID         appID
count  8.403901e+07  8.403901e+07
mean   1.400933e+06  1.389724e+03
std    8.096765e+05  8.620736e+03
min    1.000000e+00  3.540000e+02
25%    7.000290e+05  4.940000e+02
50%    1.399038e+06  5.310000e+02
75%    2.102402e+06  6.670000e+02
max    2.805117e+06  4.332690e+05


,userID,appID
46504722,1549353,9498
27485792,915323,582
70511677,2352599,496
56497484,1882690,490
40560579,1350642,1211


1.1 进行feature engineering
希望得到每个user 平均使用多少app，和使用多少种类的app

In [188]:
df_merge=pd.merge(df_user_installedapps,df_app_categories,how='inner',on='appID')

In [189]:
df_merge.head()

,userID,appID,appCategory
0,1,357,201
1,10,357,201
2,20,357,201
3,25,357,201
4,37,357,201


In [190]:
df_numofapp=pd.DataFrame({'numOfApp' : df_merge[["userID","appID"]].groupby( [ "userID"] ).size()}).reset_index()

In [191]:
df_numofapp.head()

,userID,numOfApp
0,1,79
1,9,3
2,10,97
3,12,38
4,14,62


In [192]:
df_frequency=pd.DataFrame({'number':df_merge[["userID","appCategory"]].groupby( ["userID",'appCategory'] ).size()}).reset_index()

In [193]:
df_frequency.head()

,userID,appCategory,number
0,1,0,37
1,1,103,1
2,1,108,2
3,1,201,9
4,1,203,5


In [194]:
df_frewithoutZero=df_frequency[np.all(df_frequency != 0 ,axis=1)]

In [195]:
str1="SELECT userID, appCategory, Max(number) FROM df_frewithoutZero GROUP BY userID"
df_MaxCate=pd.DataFrame(ps.sqldf(str1,locals()))

In [196]:
df_MaxCate.head()

,userID,appCategory,Max(number)
0,1,201,9
1,9,210,1
2,10,503,18
3,12,301,5
4,14,503,19


In [197]:
df_appFeature= pd.merge(df_MaxCate,df_numofapp, how='inner', on='userID')

In [198]:
df_appFeature.head()

,userID,appCategory,Max(number),numOfApp
0,1,201,9,79
1,9,210,1,3
2,10,503,18,97
3,12,301,5,38
4,14,503,19,62


1.2 处理clickTime和installedTime的关系

理论来说，应该查看clickTime是否在该用户的习惯装软件的时间范围内。
所以应该先

In [199]:
df_train = pd.read_csv('dataset/pre/train.csv')

In [200]:
df_train[df_train.conversionTime.isnull() == False].sample(20)

,label,clickTime,conversionTime,creativeID,userID,positionID,connectionType,telecomsOperator
2078103,1,251308,261028.0,376,2559184,2579,1,3
2304709,1,260945,260945.0,4228,2801298,3322,1,2
633025,1,201505,201510.0,4565,1782210,2723,1,1
1131769,1,220820,220821.0,1675,498196,5375,1,1
679077,1,201653,230858.0,2235,1986697,231,1,1
1238727,1,221530,221653.0,1004,2208342,94,1,3
744791,1,202039,202039.0,2553,2617535,5358,1,2
2057796,1,251145,261029.0,2537,471424,2579,1,1
3640668,1,301902,301902.0,3869,2735204,4753,1,3
1644925,1,232107,242159.0,1537,1352916,2623,1,1


In [201]:
df_user_appactions.sample(20)

,userID,installTime,appID
5477413,2558286,151904,771
3970355,1852882,100154,594
4906990,2289549,260203,3389
1302988,613269,100048,369
3609685,1685651,111800,1183
3254719,1521654,111508,582
3228786,1509761,262104,806
2313624,1084790,221741,536
2299587,1078264,11421,698
5266994,2457525,12227,13986


In [202]:
df_user_appactions[["userID","installTime"]].groupby(["userID"]).count().describe()

,installTime
count,781112.000000
mean,7.685800
std,11.601835
min,1.000000
25%,1.000000
50%,4.000000
75%,10.000000
max,755.000000


#### 2. 处理ad维度的feature

衡量add和app的宣传丰富程度

In [203]:
df_ad =pd.read_csv('dataset/pre/ad.csv')

In [204]:
df_ad.head()

,creativeID,adID,camgaignID,advertiserID,appID,appPlatform
0,4079,2318,147,80,14,2
1,4565,3593,632,3,465,1
2,3170,1593,205,54,389,1
3,6566,2390,205,54,389,1
4,5187,411,564,3,465,1


In [205]:
df_ad.describe()

,creativeID,adID,camgaignID,advertiserID,appID,appPlatform
count,6582.000000,6582.000000,6582.000000,6582.000000,6582.000000,6582.000000
mean,3291.500000,1786.341689,313.397144,44.381191,310.805682,1.448952
std,1900.204068,1045.890729,210.636055,24.091342,125.577377,0.497425
min,1.000000,1.000000,1.000000,1.000000,14.000000,1.000000
25%,1646.250000,882.250000,131.000000,26.000000,205.000000,1.000000
50%,3291.500000,1771.000000,274.000000,54.000000,389.000000,1.000000
75%,4936.750000,2698.750000,512.000000,57.000000,421.000000,2.000000
max,6582.000000,3616.000000,720.000000,91.000000,472.000000,2.000000


2.1 有多少素材存在于同一个ad

In [206]:
df_creaUnderAd=pd.DataFrame({"NumOfCrea_ad" : df_ad[["creativeID","adID"]].groupby(["adID"]).size()}).reset_index()

In [207]:
df_creaUnderAd.head()

,adID,NumOfCrea_ad
0,1,3
1,2,3
2,3,2
3,4,1
4,5,1


2.2 有多少素材存在于同一个广告商

In [208]:
df_creaUnderAdvertiser=pd.DataFrame({"NumOfCrea_advertiser": df_ad[["creativeID","advertiserID"]].groupby(["advertiserID"]).size()}).reset_index()

In [209]:
df_creaUnderAdvertiser.head()

,advertiserID,NumOfCrea_advertiser
0,1,5
1,2,78
2,3,56
3,4,4
4,5,1


2.3 有多少素材存在于同一个app下面

In [210]:
df_creaUnderApp=pd.DataFrame({"NumOfCrea_app": df_ad[["creativeID","appID"]].groupby(["appID"]).size()}).reset_index()

In [211]:
df_creaUnderApp.head()

,appID,NumOfCrea_app
0,14,191
1,25,10
2,68,15
3,75,11
4,83,6


2.4 有多少素材存在于同一个推广计划下面

In [212]:
df_creaUnderCam=pd.DataFrame({"NumOfCrea_Cam": df_ad[["creativeID","camgaignID"]].groupby(["camgaignID"]).size()}).reset_index()

In [213]:
df_creaUnderCam.head()

,camgaignID,NumOfCrea_Cam
0,1,8
1,2,21
2,3,1
3,4,1
4,5,4


　3。 处理user相关的feature

３.１　处理age

In [214]:
df_train = pd.read_csv('dataset/pre/train.csv')

In [215]:
df_user = pd.read_csv('dataset/pre/user.csv')

In [216]:
ageGroup_forMerge = pd.DataFrame(df_user.age.unique(),columns=[['age']])

In [217]:
ageGroup_forMerge['ageGroup'] = 0
for i in range(len(ageGroup_forMerge)):
    if 1 <= ageGroup_forMerge.age[i] < 10:
        ageGroup_forMerge.set_value(i,'ageGroup', 1)
    elif 10 <= ageGroup_forMerge.age[i] < 15:
        ageGroup_forMerge.set_value(i,'ageGroup', 2)
    elif 15 <= ageGroup_forMerge.age[i] < 20:
        ageGroup_forMerge.set_value(i,'ageGroup', 3)
    elif 20 <= ageGroup_forMerge.age[i] < 25:
        ageGroup_forMerge.set_value(i,'ageGroup', 4)
    elif 25 <= ageGroup_forMerge.age[i] < 30:
        ageGroup_forMerge.set_value(i,'ageGroup', 5)
    elif 30 <= ageGroup_forMerge.age[i] < 35:
        ageGroup_forMerge.set_value(i,'ageGroup', 6)
    elif 35 <= ageGroup_forMerge.age[i] < 40:
        ageGroup_forMerge.set_value(i,'ageGroup', 7)
    elif 40 <= ageGroup_forMerge.age[i] < 50:
        ageGroup_forMerge.set_value(i,'ageGroup', 8)
    elif 50 <= ageGroup_forMerge.age[i] < 200:
        ageGroup_forMerge.set_value(i,'ageGroup', 9)

In [218]:
ageGroup_forMerge.sample(5)

,age,ageGroup
30,7,1
8,13,2
56,55,9
60,61,9
53,49,8


３.２　处理residence

In [219]:
# residence to residenceGroup
tmp = df_user.residence.value_counts()
tmp = pd.DataFrame(df_user.residence.value_counts())
tmp.columns = [['num']]
tmp['residence'] = tmp.index
tmp['residenceGroup'] = 0
tmp.index = range(0,len(tmp))

for i in range(len(tmp)):
    if 1 <= tmp.num[i] < 1640:
        tmp.set_value(i,'residenceGroup',1)
    elif 1640 <= tmp.num[i] < 4676:
        tmp.set_value(i,'residenceGroup',2)
    elif 4676 <= tmp.num[i] < 8199:
        tmp.set_value(i,'residenceGroup',3)
    elif 8199 <= tmp.num[i] < 200000:
        tmp.set_value(i,'residenceGroup',4)

tmp = tmp.drop('num',1)


In [220]:
tmp.sample(5)

,residence,residenceGroup
156,1704,3
96,1302,4
91,402,4
288,1513,2
131,2205,3


### 4. 读取context table

In [221]:
df_position = pd.read_csv('dataset/pre/position.csv')

#### 5. 准备工作

5.1 合并所有相关的table到train里

In [222]:
#合并user相关，并处理age和residence
df_all_train = pd.merge(df_train,df_user, how='left', on='userID')
df_all_train  = pd.merge(df_all_train , ageGroup_forMerge, how='left', on='age')
df_all_train  = df_all_train.drop('age',1)
df_all_train = pd.merge(df_all_train, tmp, how='left', on='residence')
df_all_train = df_all_train.drop('residence',1)

#合并ad 以及相关feature  并remove 原始ID
df_all_train = pd.merge(df_all_train, df_ad, how='left', on='creativeID')
df_all_train = pd.merge(df_all_train, df_creaUnderAd, how='left', on='adID')
df_all_train = pd.merge(df_all_train, df_creaUnderAdvertiser, how='left', on='advertiserID')
df_all_train = pd.merge(df_all_train, df_creaUnderApp, how='left', on='appID')
df_all_train = pd.merge(df_all_train, df_creaUnderCam, how='left', on='camgaignID')
df_all_train = df_all_train.drop(["adID","advertiserID","appID","camgaignID"],1)

#合并用户app使用习惯相关table和feature
df_all_train = pd.merge(df_all_train, df_appFeature, how='left', on='userID')

#合并context到train里
df_all_train = pd.merge(df_all_train, df_position, how='left', on='positionID')

#最终删除所有无用feature
df_all_train = df_all_train.drop(["clickTime","conversionTime","hometown","appPlatform","creativeID","userID","positionID"],1)

In [223]:
df_all_train.sample(5)

,label,connectionType,telecomsOperator,gender,education,marriageStatus,haveBaby,ageGroup,residenceGroup,NumOfCrea_ad,NumOfCrea_advertiser,NumOfCrea_app,NumOfCrea_Cam,appCategory,Max(number),numOfApp,sitesetID,positionType
2286083,0,1,2,1,2,0,0,1,4,1,160,713,1,NaN,NaN,NaN,1,0
2462501,0,1,1,1,5,1,0,4,2,4,308,763,54,NaN,NaN,NaN,0,1
3244922,0,1,1,2,1,1,0,3,2,1,35,35,12,503.0,10.0,84.0,1,0
882359,0,1,1,0,3,3,0,0,2,1,35,35,12,301.0,5.0,22.0,1,0
544883,0,1,1,1,2,3,0,4,4,1,160,713,1,201.0,6.0,55.0,1,0


In [224]:
df_all_train.columns

Index(['label', 'connectionType', 'telecomsOperator', 'gender', 'education',
       'marriageStatus', 'haveBaby', 'ageGroup', 'residenceGroup',
       'NumOfCrea_ad', 'NumOfCrea_advertiser', 'NumOfCrea_app',
       'NumOfCrea_Cam', 'appCategory', 'Max(number)', 'numOfApp', 'sitesetID',
       'positionType'],
      dtype='object')

##### 5.2 用0代替所有的NaN

In [225]:
df_all_train=df_all_train.fillna(0)

In [231]:
df_all_train.sample(5)

,label,connectionType,telecomsOperator,gender,education,marriageStatus,haveBaby,ageGroup,residenceGroup,NumOfCrea_ad,NumOfCrea_advertiser,NumOfCrea_app,NumOfCrea_Cam,appCategory,Max(number),numOfApp,sitesetID,positionType
768685,0,2,1,2,4,1,0,3,4,4,56,56,4,301.0,4.0,38.0,0,1
3637003,0,1,2,2,0,1,0,3,2,1,193,193,4,0.0,0.0,0.0,0,2
562772,0,1,2,1,3,0,0,4,2,1,308,763,54,0.0,0.0,0.0,1,0
1041366,0,2,3,1,0,1,0,3,4,4,56,56,4,503.0,27.0,98.0,0,1
2583478,0,1,1,1,0,2,0,4,0,4,56,56,4,503.0,23.0,71.0,0,1


##### 5.3 用one-hot encoding处理所有的nominal feature
包括以下feature："connectionType","telecomsOperator","marriageStatus","haveBaby","residenceGroup","appCategory","sitesetID","positionType"

In [259]:
columns=["connectionType","telecomsOperator","marriageStatus","haveBaby","residenceGroup","appCategory","sitesetID","positionType"]
df_all_train

In [260]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(sparse=False)

In [278]:
df_all_train_1=df_all_train.drop(["connectionType","telecomsOperator","marriageStatus","haveBaby","residenceGroup","appCategory","sitesetID","positionType"],1)

In [279]:
for col in columns:
    enc.fit(df_all_train[[col]])
    temp=enc.transform(df_all_train[[col]])
    temp=pd.DataFrame(temp,columns=[(col+"_"+str(i)) for i in df_all_train[col]
                                    .value_counts().index])
    temp=temp.set_index(df_all_train.index.values)
    df_all_train_1=pd.concat([df_all_train_1,temp],axis=1)


In [280]:
df_all_train_1.columns

Index(['label', 'gender', 'education', 'ageGroup', 'NumOfCrea_ad',
       'NumOfCrea_advertiser', 'NumOfCrea_app', 'NumOfCrea_Cam', 'Max(number)',
       'numOfApp', 'connectionType_1', 'connectionType_2', 'connectionType_3',
       'connectionType_0', 'connectionType_4', 'telecomsOperator_1',
       'telecomsOperator_3', 'telecomsOperator_2', 'telecomsOperator_0',
       'marriageStatus_0', 'marriageStatus_2', 'marriageStatus_1',
       'marriageStatus_3', 'haveBaby_0', 'haveBaby_1', 'haveBaby_2',
       'haveBaby_3', 'haveBaby_4', 'haveBaby_5', 'haveBaby_6',
       'residenceGroup_4', 'residenceGroup_3', 'residenceGroup_2',
       'residenceGroup_0', 'residenceGroup_1', 'appCategory_0.0',
       'appCategory_503.0', 'appCategory_301.0', 'appCategory_201.0',
       'appCategory_407.0', 'appCategory_106.0', 'appCategory_402.0',
       'appCategory_209.0', 'appCategory_203.0', 'appCategory_405.0',
       'appCategory_401.0', 'appCategory_104.0', 'appCategory_105.0',
       'appCategory_

In [281]:
df_all_train_1.sample(5)

,label,gender,education,ageGroup,NumOfCrea_ad,NumOfCrea_advertiser,NumOfCrea_app,NumOfCrea_Cam,Max(number),numOfApp,...,appCategory_204.0,sitesetID_0,sitesetID_1,sitesetID_2,positionType_1,positionType_0,positionType_2,positionType_3,positionType_4,positionType_5
1728107,0,1,1,5,1,160,713,1,10.0,67.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2648357,0,2,0,4,4,56,56,4,8.0,62.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
463467,0,2,3,4,1,4,4,1,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1649642,0,0,0,9,1,35,35,12,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
680721,0,1,0,4,4,56,56,4,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


#### 6. 尝试用xgboos算法